# Modeling Fox's Wake-Up Behavior
## Table of Contents

<a id = 'imports'></a>
## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import regularizers

import matplotlib.pyplot as plt
%matplotlib inline

<a id = 'read'></a>
## Reading in Data

In [2]:
df_frames = pd.read_csv('../data/fox-falco-fd-frames.csv')
df_frames.head()

,Unnamed: 0,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,...,nfox_combo_count,nfox_dmg,nfox_direction,nfox_last_hit_by,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
1,1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
2,2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
3,3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4
4,4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,...,0,0.0,1,NaN,-60.0,10.0,60.0,322,-1.0,4


In [3]:
df_frames.drop(columns = ['Unnamed: 0', 'fox_last_hit_by', 'nfox_last_hit_by'], inplace = True)
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_Z,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4


In [4]:
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_Z,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-123,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
1,20190406T144505,-122,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
2,20190406T144505,-121,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
3,20190406T144505,-120,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4
4,20190406T144505,-119,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,10.0,60.0,322,-1.0,4


Action States 322, 323, and 324 are useless because they are the states of the game starting, so we will drop them.
Planned on removing all frames in which both characters are on the ground because. If `game` is a Game object, then I can access this information with `game.frames[frame_index].ports[active_port_index].leader.post.ground`. The problem is that this information is not tracked for the games recorded at Fight Pitt 9. I believe this information is recorded for more recent versions of Slippi.

In [5]:
useless_states = [322, 323, 324]
df_frames = df_frames.loc[(~df_frames['fox_state'].isin(useless_states)) | (~df_frames['nfox_state'].isin(useless_states))]
df_frames.reset_index(inplace = True, drop = True)
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_Z,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-49,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.872425,60.0,29,0.0,4
1,20190406T144505,-48,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.532425,60.0,29,1.0,4
2,20190406T144505,-47,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.022425,60.0,29,2.0,4
3,20190406T144505,-46,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,8.342424,60.0,29,3.0,4
4,20190406T144505,-45,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,7.492424,60.0,29,4.0,4


In [6]:
df_frames.head()

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_Z,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
0,20190406T144505,-49,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.872425,60.0,29,0.0,4
1,20190406T144505,-48,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.532425,60.0,29,1.0,4
2,20190406T144505,-47,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,9.022425,60.0,29,2.0,4
3,20190406T144505,-46,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,8.342424,60.0,29,3.0,4
4,20190406T144505,-45,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.0,1,-60.0,7.492424,60.0,29,4.0,4


In [7]:
ff_games = list(set(df_frames['game_id']))
ff_games

['20190309T012347',
 '20190406T105900',
 '20190406T114015',
 '20190406T144505',
 '20190406T102328',
 '20190406T190322',
 '20190406T104203',
 '20190406T143503',
 '20190406T214523',
 '20190406T183745',
 '20190406T114529',
 '20190406T175827',
 '20190406T190420']

In [8]:
df_frames.loc[df_frames['game_id'] == ff_games[0]]

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_Z,nfox_combo_count,nfox_dmg,nfox_direction,nfox_position_x,nfox_position_y,nfox_shield,nfox_state,nfox_state_age,nfox_stocks
23885,20190309T012347,-59,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.000000,-1,20.000000,10.636372,60.0,324,-1.0,4
23886,20190309T012347,-58,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.000000,-1,20.000000,10.593946,60.0,324,-1.0,4
23887,20190309T012347,-57,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.000000,-1,20.000000,10.551522,60.0,324,-1.0,4
23888,20190309T012347,-56,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.000000,-1,20.000000,10.509097,60.0,324,-1.0,4
23889,20190309T012347,-55,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0.000000,-1,20.000000,10.466673,60.0,324,-1.0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29245,20190309T012347,5301,0.0,0.0,0.0,0.0,0,0,0,0,...,0,1,84.259995,1,-183.147888,-129.243683,60.0,38,27.0,1
29246,20190309T012347,5302,0.0,0.0,0.0,0.0,0,0,0,0,...,0,1,84.259995,1,-183.699142,-132.343689,60.0,38,28.0,1
29247,20190309T012347,5303,0.0,0.0,0.0,0.0,0,0,0,0,...,0,1,84.259995,1,-184.319763,-135.443695,60.0,38,29.0,1
29248,20190309T012347,5304,0.0,0.0,0.0,0.0,0,0,0,0,...,0,1,84.259995,1,-185.009766,-138.543701,60.0,38,0.0,1


In [11]:
missed_tech = [183, 188, 189, 191, 196, 197]
df_frames['fox_state'] = df_frames['fox_state'].apply(lambda val: val if val not in missed_tech else 999)

In [12]:
missed_and_techs = [999, 199, 200, 201]
df_frames['target'] = df_frames['fox_state'].apply(lambda val: 0 if val not in missed_and_techs else (1 if val == 999 else (2 if val == 199 else (3 if val == 200 else 4))))

In [13]:
# class imbalance
df_frames['target'].value_counts(normalize = True)

0    0.966377
1    0.020880
2    0.005488
4    0.004415
3    0.002839
Name: target, dtype: float64

In [14]:
df_frames.columns

Index(['game_id', 'frame_index', 'fox_cstick_x', 'fox_cstick_y',
       'fox_joystick_x', 'fox_joystick_y', 'fox_Y', 'fox_X', 'fox_B', 'fox_A',
       'fox_L', 'fox_R', 'fox_Z', 'fox_combo_count', 'fox_dmg',
       'fox_direction', 'fox_position_x', 'fox_position_y', 'fox_shield',
       'fox_state', 'fox_state_age', 'fox_stocks', 'nfox_cstick_x',
       'nfox_cstick_y', 'nfox_joystick_x', 'nfox_joystick_y', 'nfox_Y',
       'nfox_X', 'nfox_B', 'nfox_A', 'nfox_L', 'nfox_R', 'nfox_Z',
       'nfox_combo_count', 'nfox_dmg', 'nfox_direction', 'nfox_position_x',
       'nfox_position_y', 'nfox_shield', 'nfox_state', 'nfox_state_age',
       'nfox_stocks', 'target'],
      dtype='object')

In [15]:
df_frames = pd.get_dummies(df_frames, columns = ['fox_state', 'nfox_state', 'fox_direction', 'nfox_direction'])

In [16]:
# Expecting 578 rows
df_frames.loc[df_frames['fox_state_199'] == 1]

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_state_440,nfox_state_441,nfox_state_442,nfox_state_443,nfox_state_445,nfox_state_446,fox_direction_-1,fox_direction_1,nfox_direction_-1,nfox_direction_1
12709,20190406T190420,4275,0.0,0.0,0.0000,0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
12710,20190406T190420,4276,0.0,0.0,0.0000,0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
12711,20190406T190420,4277,0.0,0.0,0.0000,0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
12712,20190406T190420,4278,0.0,0.0,0.0000,0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
12713,20190406T190420,4279,0.0,0.0,0.0000,0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104436,20190406T114529,8464,0.0,0.0,0.9375,0.3250,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
104437,20190406T114529,8465,0.0,0.0,0.9375,0.3250,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
104438,20190406T114529,8466,0.0,0.0,0.9375,0.3250,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
104439,20190406T114529,8467,0.0,0.0,0.9375,0.3250,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0


In [17]:
# Expecting 299 rows
df_frames.loc[df_frames['fox_state_200'] == 1]

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_state_440,nfox_state_441,nfox_state_442,nfox_state_443,nfox_state_445,nfox_state_446,fox_direction_-1,fox_direction_1,nfox_direction_-1,nfox_direction_1
9688,20190406T190420,1254,0.0,0.0,-0.9500,-0.2875,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
9689,20190406T190420,1255,0.0,0.0,-0.9625,0.0000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
9690,20190406T190420,1256,0.0,0.0,-0.9750,0.0000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
9691,20190406T190420,1257,0.0,0.0,-0.9875,0.0000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
9692,20190406T190420,1258,0.0,0.0,-0.9875,0.0000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
102463,20190406T114529,6491,0.0,0.0,0.0000,-0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
102464,20190406T114529,6492,0.0,0.0,0.0000,-0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
102465,20190406T114529,6493,0.0,0.0,0.0000,-0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
102466,20190406T114529,6494,0.0,0.0,0.0000,-0.9875,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [18]:
# Expecting 465 rows
df_frames.loc[df_frames['fox_state_201'] == 1]

,game_id,frame_index,fox_cstick_x,fox_cstick_y,fox_joystick_x,fox_joystick_y,fox_Y,fox_X,fox_B,fox_A,...,nfox_state_440,nfox_state_441,nfox_state_442,nfox_state_443,nfox_state_445,nfox_state_446,fox_direction_-1,fox_direction_1,nfox_direction_-1,nfox_direction_1
4050,20190406T144505,4001,0.0,0.0,0.9875,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4051,20190406T144505,4002,0.0,0.0,0.9875,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4052,20190406T144505,4003,0.0,0.0,0.9875,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4053,20190406T144505,4004,0.0,0.0,0.9875,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4054,20190406T144505,4005,0.0,0.0,0.9875,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94358,20190406T143503,5470,0.0,0.0,0.0000,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
94359,20190406T143503,5471,0.0,0.0,0.0000,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
94360,20190406T143503,5472,0.0,0.0,0.0000,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
94361,20190406T143503,5473,0.0,0.0,0.0000,0.0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0


In [19]:
df_frames['target'].value_counts(normalize = True)

0    0.966377
1    0.020880
2    0.005488
4    0.004415
3    0.002839
Name: target, dtype: float64

In [20]:
set(df_frames['target'])

{0, 1, 2, 3, 4}

In [21]:
set(df_frames['game_id'])

{'20190309T012347',
 '20190406T102328',
 '20190406T104203',
 '20190406T105900',
 '20190406T114015',
 '20190406T114529',
 '20190406T143503',
 '20190406T144505',
 '20190406T175827',
 '20190406T183745',
 '20190406T190322',
 '20190406T190420',
 '20190406T214523'}

In [22]:
X = df_frames.drop(columns = ['game_id', 'frame_index', 'target'])
y = to_categorical(df_frames['target'])

In [23]:
X.shape

(105316, 333)

In [ ]:
# Since I cannot set random_state to 20XX, I will set it to 2099
X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle = False, random_state = 2099)

In [ ]:
ss = StandardScaler()
Z_train = ss.fit_transform(X_train)
Z_test = ss.transform(X_test)

In [ ]:
train_sequences = TimeseriesGenerator(Z_train, y_train,
                                     length = 120, batch_size = 512)

In [ ]:
train_sequences[0][0].shape

In [ ]:
test_sequences = TimeseriesGenerator(Z_test, y_test,
                                    length = 120, batch_size = 512)

In [ ]:
Z_train.shape

In [ ]:
model = Sequential()

model.add(GRU(337, input_shape = (120, 337), return_sequences = True))
model.add(GRU(337, return_sequences = False))

model.add(Dense(128, activation = 'relu'))

model.add(Dense(5, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

In [ ]:
hist = model.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('No Regularization');
plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy');
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue');
plt.legend();

In [ ]:
# EARLY STOPPING
model_es = Sequential()

model_es.add(GRU(337, input_shape = (120, 337), return_sequences = True))
model_es.add(GRU(337, return_sequences = False))

model_es.add(Dense(128, activation = 'relu'))

model_es.add(Dense(5, activation = 'softmax'))

early_stop = EarlyStopping(monitor = 'val_loss', min_delta = 0, patience = 5, verbose = 1, mode = 'auto')
model_es.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'], callbacks = early_stop)

hist_es = model_es.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization and Early Stopping')
plt.title('Not expecting much difference')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'red')
plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

plt.legend();

In [ ]:
# L2
model_l2 = Sequential()

model_l2.add(GRU(337, input_shape = (120, 337), return_sequences = True, kernel_regularizer = regularizers.l2(0.01)))
model_l2.add(GRU(337, return_sequences = False, kernel_regularizer = regularizers.l2(0.01)))

model_l2.add(Dense(128, activation = 'relu', kernel_regularizer = regularizers.l2(0.01)))

model_l2.add(Dense(5, activation = 'softmax', kernel_regularizer = regularizers.l2(0.01)))

model_l2.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

hist_l2 = model_l2.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization, Early Stopping, and L2 Regularization')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

# plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'darkred')
# plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

plt.plot(hist_l2.history['loss'], label = 'Train Loss L2', color = 'orange')
plt.plot(hist_l2.history['val_loss'], label = 'Test Loss L2', color = 'yellow')

plt.legend();

In [ ]:
# Dropout
model_do = Sequential()

model_do.add(GRU(337, input_shape = (120, 337), return_sequences = True))
model_do.add(GRU(337, return_sequences = False))

model_do.add(Dense(128, activation = 'relu'))
model_do.add(Dropout(0.5))

model_do.add(Dense(5, activation = 'softmax'))

model_do.compile(loss = 'categorical_crossentropy', optimizer = Adam(0.001), metrics = ['accuracy'])

hist_do = model_do.fit_generator(train_sequences, epochs = 5, validation_data = test_sequences)

In [ ]:
plt.title('Comparing No Regularization, Early Stopping, L2 Regularization, and Dropout')

plt.plot(hist.history['loss'], label = 'Train Loss', color = 'navy')
plt.plot(hist.history['val_loss'], label = 'Test Loss', color = 'skyblue')

# plt.plot(hist_es.history['loss'], label = 'Train Loss ES', color = 'darkred')
# plt.plot(hist_es.history['val_loss'], label = 'Test Loss ES', color = 'pink')

# plt.plot(hist_l2.history['loss'], label = 'Train Loss L2', color = 'orange')
# plt.plot(hist_l2.history['val_loss'], label = 'Test Loss L2', color = 'yellow')

plt.plot(hist_do.history['loss'], label = 'Train Loss Dropout', color = 'darkgreen')
plt.plot(hist_do.history['val_loss'], label = 'Test Loss Dropout', color = 'lightgreen')

plt.legend();

In [ ]:
plt.plot(hist_do.history['loss'], label = 'Train Loss Dropout', color = 'darkgreen')
plt.plot(hist_do.history['val_loss'], label = 'Test Loss Dropout', color = 'lightgreen')